#### 1. 挖掘数据定义  
遍历每一列, 阅读没类的描述确保理解这些数据表示的侧面, 并使用统计和绘图遍历列  
可以查看其它data scientist的工作来理解每一列
#### 2. 选择一个metric
#### 3. Data Exploration and Data Cleaning  
1. 此阶段要处理异常值和缺失值, 之后才能建模  
2. 先查看数据标签的分布, 看数据是否平衡  
   如果变迁季度不平衡, 就要注意在交叉验证时使用分层抽样  
3. 不同的数据类型表示不同的统计类型  
    1. float可能是连续值  
    2. int可能是boolean型或离散的有序变量    
       可通过查看int型列的值个数确定; train.select_dtypes('object').count_values()
    3. object/categorical可能是标签  
       train.select_dtypes('object').head()
4. 缺失值处理  
   计算每一列缺失值比例  
    ```python
    import pandas as pd
    # Number of missing in each column
    missing = pd.DataFrame(data.isnull().sum()).rename(columns = {0: 'total'})
    # Create a percentage missing
    missing['percent'] = missing['total'] / len(data) 
    ```
    
#### 4. Feature Engineering  
分为手动特征工程和自动特征工程(featuretools,十分之一的时间却有更好的效果)
手动特征工程:  
1. 首先, 根据id查看每一列的统计属性   
```python
# Aggregate individual data for each household
id_agg = ind.groupby('id').agg(['min', 'max', 'mean', 'sum'])  
这会多出来4倍的统计属性, 而且多数存在线性相关性 , 因此需要做特征选择  
```
2. 其次, 可以利用领域知识构建新特征  
```python
# No toilet, no electricity, no floor, no water service, no ceiling
house['warning'] = 1 * (house['sanitario1'] + 
                        (house['elec'] == 0) + 
                        house['pisonotiene'] + 
                        house['abastaguano'] + 
                        (house['cielorazo'] == 0))
```  
最好的方法是手动增加新特征后,再使用feature tools进行自动特征

#### 5. Feature Selection  
一种方法是查看两个特征之间的相关性, 相关系数高的叫做高线性相关
```python
import numpy as np

threshold = 0.95

# Create correlation matrix
corr_matrix = data.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(abs(upper[column]) > threshold)]

data = data.drop(columns = to_drop)

还可以通过模型, 如随机森林来选择特征
```